## 1. Feature Evaluation

In this file, we will be evaluating features. The procedure is as follow,

1. Subject-Dependent evaluation: build model for each subject
2. 10-CrossValidation: with 75-25 train-test ration
3. `Trial-dependent`: Segment First then Split
4. The DEAP data set contain 63 seconds long trial, discard the first 3 seconds and segment the trial into 60 samples. (1 second long)

Plan 1
====

In Sklearn, there is a function `sklearn.model_selection.StratifiedShuffleSplit`.

1. Stratified: ensure label ratio
2. ShuffleSplit: this is not K-fold. Just random all segment with the ability to pick train-test.

In [1]:
import time
import numpy as np
from tqdm.notebook import tqdm

from components.dataset_jo import *

dataset = Dataset_subjectDependent('data')
dataset.set_segment(60)
for filename in dataset.get_file_list():
    data, labels, _ = dataset.get_data(filename, return_type='numpy')
    print(filename, labels.shape, labels.sum(axis=0)/labels.shape[0]*100)

Found: 32 files
s01 (2400, 2) [47.5 60. ]
s02 (2400, 2) [55. 60.]
s03 (2400, 2) [55. 20.]
s04 (2400, 2) [40. 40.]
s05 (2400, 2) [60.  47.5]
s06 (2400, 2) [75.  42.5]
s07 (2400, 2) [70.  62.5]
s08 (2400, 2) [55.  57.5]
s09 (2400, 2) [50. 60.]
s10 (2400, 2) [50. 55.]
s11 (2400, 2) [60.  37.5]
s12 (2400, 2) [52.5 82.5]
s13 (2400, 2) [42.5 85. ]
s14 (2400, 2) [50.  67.5]
s15 (2400, 2) [50.  47.5]
s16 (2400, 2) [37.5 50. ]
s17 (2400, 2) [55. 60.]
s18 (2400, 2) [60.  62.5]
s19 (2400, 2) [57.5 67.5]
s20 (2400, 2) [57.5 77.5]
s21 (2400, 2) [52.5 80. ]
s22 (2400, 2) [45. 60.]
s23 (2400, 2) [65. 25.]
s24 (2400, 2) [45.  82.5]
s25 (2400, 2) [47.5 72.5]
s26 (2400, 2) [65.  42.5]
s27 (2400, 2) [75.  67.5]
s28 (2400, 2) [62.5 45. ]
s29 (2400, 2) [57.5 62.5]
s30 (2400, 2) [67.5 47.5]
s31 (2400, 2) [57.5 47.5]
s32 (2400, 2) [50.  67.5]


In [2]:
from sklearn.model_selection import GroupShuffleSplit, StratifiedGroupKFold,StratifiedShuffleSplit 
print("===== GroupShuffleSplit =====")
X,y,groups = dataset.get_data('s01', Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
print(X.shape, y.squeeze().shape, groups.shape)
cv = GroupShuffleSplit(n_splits=10, train_size=0.75, random_state=0)
for train_idxs, test_idxs in cv.split(X, y.squeeze(), groups):
    print(train_idxs.shape, test_idxs.shape, set(groups[train_idxs]).intersection(groups[test_idxs]) )
    print(sum(y[train_idxs])/len(train_idxs),sum(y[test_idxs])/len(test_idxs))

print()

print("===== StratifiedGroupKFold =====")
X,y,groups = dataset.get_data('s01', Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
print(X.shape, y.squeeze().shape, groups.shape)
cv = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=0)
for train_idxs, test_idxs in cv.split(X, y.squeeze(), groups):
    print(train_idxs.shape, test_idxs.shape, set(groups[train_idxs]).intersection(groups[test_idxs]) )
    print(sum(y[train_idxs])/len(train_idxs),sum(y[test_idxs])/len(test_idxs))

print()

print("===== StratifiedShuffleSplit =====")
X,y,groups = dataset.get_data('s01', Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
print(X.shape, y.squeeze().shape, groups.shape)
cv = StratifiedShuffleSplit(n_splits=10, train_size=0.75, random_state=0)
for train_idxs, test_idxs in cv.split(X, y.squeeze(), groups):
    print(train_idxs.shape, test_idxs.shape, set(groups[train_idxs]).intersection(groups[test_idxs]) )
    print(sum(y[train_idxs])/len(train_idxs),sum(y[test_idxs])/len(test_idxs))

===== GroupShuffleSplit =====
(2400, 32, 128) (2400,) (2400,)
(1800,) (600,) set()
[0.66666667] [0.4]
(1800,) (600,) set()
[0.56666667] [0.7]
(1800,) (600,) set()
[0.63333333] [0.5]
(1800,) (600,) set()
[0.66666667] [0.4]
(1800,) (600,) set()
[0.63333333] [0.5]
(1800,) (600,) set()
[0.6] [0.6]
(1800,) (600,) set()
[0.53333333] [0.8]
(1800,) (600,) set()
[0.7] [0.3]
(1800,) (600,) set()
[0.56666667] [0.7]
(1800,) (600,) set()
[0.56666667] [0.7]

===== StratifiedGroupKFold =====
(2400, 32, 128) (2400,) (2400,)
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.58333333] [0.75]
(2160,) (240,) set()
[0.55555556] [1.]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.58333333] [0.75]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.61111111] [0.5]
(2160,) (240,) set()
[0.61111111] [0.5]

===== StratifiedShuffleSplit =====
(2400, 32, 128) (2400,) (2400,)
(1800,) (600,) {0

As a result, I will use `GroupShuffleSplit`.

In [3]:
def train_model(X_ori,y_ori,groups_ori,filename=None, kernel='rbf'):
    
    # Make a copy because I am paranoid
    X,y,groups = X_ori.copy(), y_ori.copy(), groups_ori.copy()

    from sklearn.svm import SVC
    from sklearn.model_selection import StratifiedShuffleSplit 
    from sklearn.model_selection import cross_val_score

    model = SVC(kernel=kernel,max_iter=50000)
    cv = StratifiedShuffleSplit(n_splits=10, train_size=0.75, random_state=0)
    cross = cross_val_score(model, X, y, cv=cv, n_jobs=8)
    
    # We probably dont need this
    model = SVC(kernel=kernel, max_iter=50000)
    model.fit(X, y)
    ans = model.predict(X)
    acc = sum(ans == y) / len(y)
    # If the model answer with all 0 or 1, we print this message
    if( sum(ans) == len(y) or sum(ans) == 0 ): 
        print(f"-----WARNING: Model {filename} failed to learn: sum(ans)={sum(ans)} sum(y)={sum(y)} len(y)={len(y)}")
    return model, acc, cross

## 2. Spectral Features

In [5]:
from mne_features.feature_extraction import FeatureExtractor

In [8]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

accs,cv_means,cv_stds  = [],[],[]
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

accs_v,cv_means_v,cv_stds_v  = [],[],[]
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
    # count+=1
    # if(count == 5): break
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

	AROUSAL-s01|Acc=0.76917|10-CV=0.68417|STD=0.01613|Time spend=2.089592695236206
	AROUSAL-s02|Acc=0.64292|10-CV=0.62183|STD=0.00643|Time spend=2.206979274749756
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=1.6508989334106445
	AROUSAL-s04|Acc=0.65292|10-CV=0.6185|STD=0.01223|Time spend=2.1168110370635986
	AROUSAL-s05|Acc=0.63667|10-CV=0.57633|STD=0.01115|Time spend=2.2953717708587646
	AROUSAL-s06|Acc=0.63292|10-CV=0.59583|STD=0.01847|Time spend=2.158255100250244
	AROUSAL-s07|Acc=0.72833|10-CV=0.66967|STD=0.01021|Time spend=2.0664174556732178
	AROUSAL-s08|Acc=0.72417|10-CV=0.64083|STD=0.00851|Time spend=2.1546568870544434
	AROUSAL-s09|Acc=0.67792|10-CV=0.63567|STD=0.00692|Time spend=2.0801613330841064
	AROUSAL-s10|Acc=0.75958|10-CV=0.69983|STD=0.01015|Time spend=2.1571784019470215
	AROUSAL-s11|Acc=0.62917|10-CV=0.62683|STD=0.00174|Time spend=2.063520669937134
-----WARNING: Model s12 failed to learn: sum(an

## 3. Asymetry

In [9]:
channels = ['Fp1','AF3','F3','F7','FC5','FC1','C3','T7','CP5','CP1','P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4','F8','FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
left_channels = ['Fp1','F7','F3','T7','P7','C3','P3','O1','AF3','FC5','FC1','CP5','CP1','PO3']
right_channels = ['Fp2','F8','F4','T8','P8','C4','P4','O2','AF4','FC6','FC2','CP6','CP2','PO4']
left_channel_indexes = [ channels.index(ch) for ch in left_channels ]
right_channel_indexes = [ channels.index(ch) for ch in right_channels ]

print(f"{left_channel_indexes=}")
print(f"{right_channel_indexes=}")

frontal_channels = ['FC5','FC1','FC2','FC6','F7','F3','Fz','F4','F8','Fp1','Fp2']
posterior_channels = ['CP5','CP1','CP2','CP6','P7','P3','Pz','P4','P8','O1','O2']

frontal_channel_indexes = [ channels.index(ch) for ch in frontal_channels ]
posterior_channel_indexes = [ channels.index(ch) for ch in posterior_channels ]

print(f"{frontal_channel_indexes=}")
print(f"{posterior_channel_indexes=}")

left_channel_indexes=[0, 3, 2, 7, 11, 6, 10, 13, 1, 4, 5, 8, 9, 12]
right_channel_indexes=[16, 20, 19, 25, 29, 24, 28, 31, 17, 21, 22, 26, 27, 30]
frontal_channel_indexes=[4, 5, 22, 21, 3, 2, 18, 19, 20, 0, 16]
posterior_channel_indexes=[8, 9, 27, 26, 11, 10, 15, 28, 29, 13, 31]


In [10]:
bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

accs,cv_means,cv_stds  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count = 0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tAROUSAL-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs[feature_name].append(acc)
        cv_means[feature_name].append(cross.mean())
        cv_stds[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"AROUSAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

accs_v,cv_means_v,cv_stds_v  = {'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]},{'DASM':[],'RASM':[],'DCAU':[]}
# count=0
for filename in dataset.get_file_list():
    start = time.time()
    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=data)

    PSD_left = X[:, left_channel_indexes].copy()
    PSD_right = X[:, right_channel_indexes].copy()
    PSD_frontal = X[:, frontal_channel_indexes].copy()
    PSD_posterior = X[:, posterior_channel_indexes].copy()
    X = {
        'DASM': PSD_left - PSD_right,
        'RASM': PSD_left / PSD_right,
        'DCAU': PSD_frontal - PSD_posterior,
    }

    for feature_name in ['DASM','RASM','DCAU']:
        _, acc, cross = train_model(X[feature_name], labels.reshape(-1), groups, filename=filename)
        print(f"\tVALENCE-{filename}-{feature_name}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
        accs_v[feature_name].append(acc)
        cv_means_v[feature_name].append(cross.mean())
        cv_stds_v[feature_name].append(cross.std())
    # count+=1
    # if(count == 3): break
for feature_name in ['DASM','RASM','DCAU']:
    print(f"VALENCE-{feature_name}|Acc={sum(accs_v[feature_name])/len(accs_v[feature_name])}|10-CV={sum(cv_means_v[feature_name])/len(cv_means_v[feature_name])}|STD={sum(cv_stds_v[feature_name])/len(cv_stds_v[feature_name])}")

for feature_name in ['DASM','RASM','DCAU']:
    accs[feature_name].extend(accs_v[feature_name])
    cv_means[feature_name].extend(cv_means_v[feature_name])
    cv_stds[feature_name].extend(cv_stds_v[feature_name])
    print(f"TOTAL-{feature_name}|Acc={sum(accs[feature_name])/len(accs[feature_name])}|10-CV={sum(cv_means[feature_name])/len(cv_means[feature_name])}|STD={sum(cv_stds[feature_name])/len(cv_stds[feature_name])}")

	AROUSAL-s01-DASM|Acc=0.61|10-CV=0.60083|STD=0.00436|Time spend=1.019376516342163
	AROUSAL-s01-RASM|Acc=0.60833|10-CV=0.60117|STD=0.00259|Time spend=1.5545539855957031
	AROUSAL-s01-DCAU|Acc=0.63167|10-CV=0.61517|STD=0.00762|Time spend=2.0637810230255127
	AROUSAL-s02-DASM|Acc=0.615|10-CV=0.60217|STD=0.00478|Time spend=1.122661828994751
	AROUSAL-s02-RASM|Acc=0.60708|10-CV=0.59767|STD=0.00359|Time spend=1.6134107112884521
	AROUSAL-s02-DCAU|Acc=0.61583|10-CV=0.60417|STD=0.00629|Time spend=2.0919063091278076
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03-DASM|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=0.9721813201904297
	AROUSAL-s03-RASM|Acc=0.80208|10-CV=0.7995|STD=0.0013|Time spend=1.2753639221191406
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03-DCAU|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=1.804645299911499
	AROUSAL-s04-DASM|Acc=0.63333|10-CV=0.618|STD=0.00657|Time spend=1.086745262145996
	AROUSAL-s04-RASM|A

## 4. Connectivity

### 4.1 coherence

In [11]:
from mne_connectivity import spectral_connectivity_time

def save(data,filename):
    with open(f'cache/{filename}.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
    with open(f'cache/{filename}.pickle', 'rb') as handle:
        data = pickle.load(handle)
    return data

In [ ]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
for filename in tqdm(dataset.get_file_list()):
    # if(filename in ['s01','s02','s03','s04','s05','s06','s07']): continue
    print("----"*5, filename, "----"*5)
    # start = time.time()
    epochs, _, _ = dataset.get_data(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne')
    # epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in range(0,len(epochs), step):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='coh',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoch, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_COH_AROUSAL_{filename}")

    epochs, _, _ = dataset.get_data(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne')
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in tqdm(range(0,len(epochs), step)):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='coh',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoc
        # h, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_COH_VALENCE_{filename}")

In [12]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_COH_AROUSAL_{filename}")
    assert X.shape == (2400,2112)
    _, labels,groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	AROUSAL-s01|Acc=0.77917|10-CV=0.66467|STD=0.01356|Time spend=32.63673949241638
	AROUSAL-s02|Acc=0.70292|10-CV=0.64933|STD=0.01033|Time spend=32.94605565071106
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=22.42845129966736
	AROUSAL-s04|Acc=0.73042|10-CV=0.6685|STD=0.01294|Time spend=31.360233545303345
	AROUSAL-s05|Acc=0.72083|10-CV=0.6295|STD=0.01123|Time spend=35.94806909561157
	AROUSAL-s06|Acc=0.67792|10-CV=0.6125|STD=0.01502|Time spend=34.917884349823
	AROUSAL-s07|Acc=0.76|10-CV=0.67683|STD=0.01403|Time spend=31.535387754440308
	AROUSAL-s08|Acc=0.76417|10-CV=0.653|STD=0.01145|Time spend=36.99915051460266
	AROUSAL-s09|Acc=0.75417|10-CV=0.685|STD=0.01638|Time spend=34.030192136764526
	AROUSAL-s10|Acc=0.81292|10-CV=0.73817|STD=0.01301|Time spend=33.61989498138428
	AROUSAL-s11|Acc=0.67542|10-CV=0.647|STD=0.01108|Time spend=35.153825521469116
-----WARNING: Model s12 failed to learn: sum(ans)=2400.0 sum(y)

In [13]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_COH_VALENCE_{filename}")
    assert X.shape == (2400,2112)
    _, labels,groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    _, acc, cross = train_model(X, labels.reshape(-1), groups,filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)
print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	VALENCE-s01|Acc=0.72|10-CV=0.56933|STD=0.01601|Time spend=37.02593231201172
	VALENCE-s02|Acc=0.7475|10-CV=0.70867|STD=0.00933|Time spend=24.315701246261597
	VALENCE-s03|Acc=0.75333|10-CV=0.67583|STD=0.02042|Time spend=34.49039268493652
	VALENCE-s04|Acc=0.77125|10-CV=0.70467|STD=0.01631|Time spend=31.66639995574951
	VALENCE-s05|Acc=0.71875|10-CV=0.6625|STD=0.01109|Time spend=36.92310857772827
-----WARNING: Model s06 failed to learn: sum(ans)=2400.0 sum(y)=1800.0 len(y)=2400
	VALENCE-s06|Acc=0.75|10-CV=0.75|STD=0.0|Time spend=32.14384746551514
	VALENCE-s07|Acc=0.76583|10-CV=0.73417|STD=0.00544|Time spend=29.645787954330444
	VALENCE-s08|Acc=0.74125|10-CV=0.631|STD=0.01846|Time spend=37.00292706489563
	VALENCE-s09|Acc=0.78542|10-CV=0.75083|STD=0.02|Time spend=30.02628469467163
	VALENCE-s10|Acc=0.85083|10-CV=0.813|STD=0.0177|Time spend=24.604620695114136
	VALENCE-s11|Acc=0.68708|10-CV=0.6495|STD=0.01432|Time spend=33.66275978088379
	VALENCE-s12|Acc=0.75958|10-CV=0.65067|STD=0.0092|Time spe

### Connectivity PLV

In [ ]:
# bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
for filename in tqdm(dataset.get_file_list()):
    # if(filename in ['s01','s02','s03','s04','s05','s06','s07']): continue
    print("----"*5, filename, "----"*5)
    # start = time.time()
    epochs, _, _ = dataset.get_data(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='mne')
    # epochs, _ = dataset.get_data_mne_epochs(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, split=False)
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in range(0,len(epochs), step):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='plv',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoch, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_PLV_AROUSAL_{filename}_1s_trial_depen")

    epochs, _, _ = dataset.get_data(filename, sfreq=128, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='mne')
    # X = fe.fit_transform(X=data)
    # print(epochs, labels.shape)
    step = 150//2
    X = []
    # count = 0
    for index in tqdm(range(0,len(epochs), step)):
        # print(index,index+step)
        conn = spectral_connectivity_time(epochs[index:index+step], method='plv',mode='multitaper',sfreq=128, freqs=range(7,64), verbose='CRITICAL')
        # (step, 528, 57, 128) => (epoc
        # h, conn, freq, time) .mean(axis=3)
        # (step, 528, 57) => (epoch, conn, freq)
        # print(conn.get_data('raveled').mean(axis=3).shape)
        conn = conn.get_data('raveled').mean(axis=3)
        # (step, 528*4 ) =>(step, 2112 )
        x = np.concatenate([ conn[:,:,7-7:8-7 ].mean(axis=2),
                            conn[:,:,8-7:12-7 ].mean(axis=2),
                            conn[:,:,12-7:30-7 ].mean(axis=2),
                            conn[:,:,30-7:64-7 ].mean(axis=2) ],axis=1)
        X.append(x)
        # count += 1
        # if(count == 3): break
    X = np.vstack(X)
    print(X.shape)
    save(X,f"X_PLV_VALENCE_{filename}_1s_trial_depen")

In [14]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_PLV_AROUSAL_{filename}")
    assert X.shape == (2400,2112)
    _, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	AROUSAL-s01|Acc=0.75542|10-CV=0.6555|STD=0.01295|Time spend=37.43366312980652
	AROUSAL-s02|Acc=0.71083|10-CV=0.64733|STD=0.01083|Time spend=37.73756408691406
-----WARNING: Model s03 failed to learn: sum(ans)=0.0 sum(y)=480.0 len(y)=2400
	AROUSAL-s03|Acc=0.8|10-CV=0.8|STD=0.0|Time spend=28.663148641586304
	AROUSAL-s04|Acc=0.7325|10-CV=0.6655|STD=0.0182|Time spend=36.61770296096802
	AROUSAL-s05|Acc=0.72875|10-CV=0.62983|STD=0.01074|Time spend=41.52228379249573
	AROUSAL-s06|Acc=0.67542|10-CV=0.608|STD=0.01628|Time spend=39.19609069824219
	AROUSAL-s07|Acc=0.75375|10-CV=0.67167|STD=0.00989|Time spend=37.3769097328186
	AROUSAL-s08|Acc=0.75625|10-CV=0.64217|STD=0.01276|Time spend=41.02107501029968
	AROUSAL-s09|Acc=0.75542|10-CV=0.68333|STD=0.01698|Time spend=36.57077383995056
	AROUSAL-s10|Acc=0.80958|10-CV=0.72717|STD=0.01106|Time spend=34.97941851615906
	AROUSAL-s11|Acc=0.6925|10-CV=0.65017|STD=0.01248|Time spend=35.66855216026306
-----WARNING: Model s12 failed to learn: sum(ans)=2400.0 sum

In [15]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):
    start = time.time()
    X = load(f"X_PLV_VALENCE_{filename}")
    assert X.shape == (2400,2112)
    _, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)

print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

	VALENCE-s01|Acc=0.71708|10-CV=0.574|STD=0.01315|Time spend=41.36754584312439
	VALENCE-s02|Acc=0.75208|10-CV=0.706|STD=0.01123|Time spend=31.821028470993042
	VALENCE-s03|Acc=0.7575|10-CV=0.67217|STD=0.02378|Time spend=36.953999042510986
	VALENCE-s04|Acc=0.77292|10-CV=0.6935|STD=0.01651|Time spend=32.18308210372925
	VALENCE-s05|Acc=0.72583|10-CV=0.6655|STD=0.01183|Time spend=36.31413412094116
-----WARNING: Model s06 failed to learn: sum(ans)=2400.0 sum(y)=1800.0 len(y)=2400
	VALENCE-s06|Acc=0.75|10-CV=0.75|STD=0.0|Time spend=32.59681844711304
	VALENCE-s07|Acc=0.75958|10-CV=0.72567|STD=0.00559|Time spend=30.242356538772583
	VALENCE-s08|Acc=0.75125|10-CV=0.62717|STD=0.01625|Time spend=38.52914834022522
	VALENCE-s09|Acc=0.79333|10-CV=0.74683|STD=0.01702|Time spend=31.223616123199463
	VALENCE-s10|Acc=0.84917|10-CV=0.81033|STD=0.01552|Time spend=25.97879695892334
	VALENCE-s11|Acc=0.70208|10-CV=0.65217|STD=0.01323|Time spend=36.84693932533264
	VALENCE-s12|Acc=0.76083|10-CV=0.64983|STD=0.01045

## 5. CSP

In [16]:
from components.CSP import CSP
from mne_features.feature_extraction import FeatureExtractor

bands = [(0,4), (4,8), (8,12), (12,30), (30,64)]
# [alias_feature_function]__[optional_param]
params = dict({
    'pow_freq_bands__log':True,
    'pow_freq_bands__normalize':False,
    'pow_freq_bands__freq_bands':bands
})

In [17]:
accs, cv_means, cv_stds = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_AROUSAL, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape == (2400,160)
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tAROUSAL-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs.append(acc)
    cv_means.append(cross.mean())
    cv_stds.append(cross.std())
print(f"AROUSAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s01|Acc=0.72458|10-CV=0.71467|STD=0.01157|Time spend=2.7334656715393066


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s02|Acc=1.0|10-CV=1.0|STD=0.0|Time spend=0.9059147834777832


...

	AROUSAL-s30|Acc=0.83833|10-CV=0.83133|STD=0.01282|Time spend=1.844303846359253


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s31|Acc=0.94958|10-CV=0.94733|STD=0.00797|Time spend=1.3076021671295166


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	AROUSAL-s32|Acc=0.8275|10-CV=0.826|STD=0.01225|Time spend=1.7909026145935059
AROUSAL|Acc=0.8031380208333333|10-CV=0.796890625|STD=0.008413643019671204


In [18]:
accs_v, cv_means_v, cv_stds_v = [],[],[]
for filename in tqdm(dataset.get_file_list()):

    data, labels, groups = dataset.get_data(filename, stimuli=Dataset_subjectDependent.STIMULI_VALENCE, return_type='numpy')
    labels = labels.squeeze()
    start = time.time()
    filters = CSP(data[labels==0], data[labels==1])
    
    csp_data = []
    for index in range(len(data)):
        
        # (672, 32) @ (32, 32) = (672, 32)
        csp = data[index].T @ filters[labels[index].astype(int)]
        # (1, 32, 672)
        csp = np.expand_dims(csp.T, axis=0)
        csp_data.append(csp)

    csp_data = np.vstack(csp_data)
    # print(csp_data.shape) => (2400, 32, 128)
    fe = FeatureExtractor(sfreq=128, selected_funcs=['pow_freq_bands'],params=params,n_jobs=8)
    X = fe.fit_transform(X=csp_data)
    # print(X.shape, time.time() - start) => (2400, 160) 1.2374355792999268

    assert X.shape == (2400,160)
    _, acc, cross = train_model(X, labels.reshape(-1), groups, filename=filename)
    print(f"\tVALENCE-{filename}|Acc={round(acc,5)}|10-CV={round(cross.mean(),5)}|STD={round(cross.std(),5)}|Time spend={time.time() - start}")
    accs_v.append(acc)
    cv_means_v.append(cross.mean())
    cv_stds_v.append(cross.std())
print(f"VALENCE|Acc={sum(accs_v)/len(accs_v)}|10-CV={sum(cv_means_v)/len(cv_means_v)}|STD={sum(cv_stds_v)/len(cv_stds_v)}")

  0%|          | 0/32 [00:00<?, ?it/s]

/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s01|Acc=0.55375|10-CV=0.53983|STD=0.01097|Time spend=2.6032896041870117


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s02|Acc=0.99667|10-CV=0.99633|STD=0.00163|Time spend=0.9560387134552002


...

	VALENCE-s30|Acc=0.84958|10-CV=0.84717|STD=0.01346|Time spend=1.6595885753631592


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s31|Acc=0.58833|10-CV=0.58033|STD=0.00557|Time spend=2.654433012008667


/root/projects/components/CSP.py:78: ComplexWarning: Casting complex values to real discards the imaginary part
  return SFa.astype(np.float32)


	VALENCE-s32|Acc=0.57417|10-CV=0.55367|STD=0.01511|Time spend=2.51802659034729
VALENCE|Acc=0.7379557291666665|10-CV=0.7289114583333332|STD=0.010317328234950098


In [19]:
accs.extend(accs_v)
cv_means.extend(cv_means_v)
cv_stds.extend(cv_stds_v)

print(f"TOTAL|Acc={sum(accs)/len(accs)}|10-CV={sum(cv_means)/len(cv_means)}|STD={sum(cv_stds)/len(cv_stds)}")

TOTAL|Acc=0.770546875|10-CV=0.7629010416666667|STD=0.009365485627310651
